In [ ]:
import numpy as np
import pandas as pd
import k3d

from ipywidgets import interact, Output
from ipywidgets.widgets import FloatSlider, IntSlider, Button, RadioButtons
from ipywidgets.widgets import HBox, interactive_output, VBox, Layout
from IPython.display import display, clear_output

err = np.arange(3)*np.finfo(dtype=np.float32).resolution

def xyz(x1, y1, z1, x2, y2, z2, x3, y3, z3, radio):
    pos1 = np.array([x1, y1, z1], dtype=np.float32)
    pos2 = np.array([x2, y2, z2], dtype=np.float32)
    pos3 = np.array([x3, y3, z3], dtype=np.float32)
    
    
    point1.positions = pos1
    point2.positions = pos2
    point3.positions = pos3 
    
    three_points = np.float32([point1.positions, point2.positions, point3.positions])
    x, y, z = three_points[:,0]+err, three_points[:,1]+err, three_points[:,2]+err
    X, Y, Z = data_points.positions[:,0], data_points.positions[:,1], data_points.positions[:,2]
    
    M = np.vstack([x, y, np.ones(3)]).T
    try:
        A, B, D = np.matmul(np.linalg.inv(M), z).round(10)

        if radio == 'below':
            cond = np.vstack([X, Y, Z]).T < np.vstack([X, Y, A*X+B*Y+D]).T
        if radio == 'above':
            cond = np.vstack([X, Y, Z]).T > np.vstack([X, Y, A*X+B*Y+D]).T

        data_points.colors = np.zeros(data.shape[0], dtype=np.uint32)
        data_points.colors[cond[:,0]+cond[:,1]+cond[:,2]] = 0xff0000
        data_points.colors = data_points.colors
        plane.vertices = np.float32([pos1, pos2, pos3])
        return cond
    
    except np.linalg.LinAlgError:
        plane.vertices = np.float32([pos1, pos2, pos3])

        
        
    
def on_button_clicked(_):
    with out:
        cond = xyz(x1.value, y1.value, z1.value, x2.value, y2.value, z2.value, x3.value, y3.value, z3.value, radio.value)
        clear_output()
        cutted_data.append(data[cond[:,0]+cond[:,1]+cond[:,2]])
        print('Cutted!')

In [ ]:
X,Y,Z = np.meshgrid(np.linspace(0,4,7), np.linspace(0,4,7), np.linspace(0,4,7))
data = np.vstack([np.ravel(X), np.ravel(Y), np.ravel(Z)]).T
data += np.random.uniform(-0.05, 0.05, data.shape)

In [ ]:
cutted_data = []
origin = np.array([data[:,0].min(), data[:,1].min(), data[:,2].min()])
end = np.array([data[:,0].max(), data[:,1].max(), data[:,2].max()])

In [ ]:
# visualisation widget
plot = k3d.plot()

point1 = k3d.points(origin.astype(np.float32), point_size=0.06, color=0x00ff00)
point2 = k3d.points((end+origin).astype(np.float32), point_size=0.06, color=0x00ff00)
point3 = k3d.points(end.astype(np.float32), point_size=0.06, color=0x00ff00)

plane = k3d.mesh([point1.positions, point2.positions, point3.positions], [0,1,2])

data_points = k3d.points(data.astype(np.float32), point_size=0.04, color=0) 
origin_point = k3d.points(origin.astype(np.float32), point_size=0.04)
end_point = k3d.points(end.astype(np.float32), point_size=0.04)

plot += point1 + point2 + point3 + plane + data_points + origin_point + end_point

In [ ]:
# widget interface
x1 = FloatSlider(value=origin[0], min=origin[0], max=end[0], step=0.01, description='x1')
y1 = FloatSlider(value=origin[1], min=origin[1], max=end[1],step=0.01, description='y1')
z1 = FloatSlider(value=origin[2], min=origin[2], max=end[2], step=0.01, description='z1')

x2 = FloatSlider(value=origin[0]+end[0], min=origin[0], max=end[0], step=0.01, description='x2')
y2 = FloatSlider(value=origin[1], min=origin[1], max=end[1],step=0.01, description='y2')
z2 = FloatSlider(value=origin[2]+end[2], min=origin[2], max=end[2], step=0.01, description='z2')

x3 = FloatSlider(value=end[0], min=origin[0], max=end[0], step=0.01, description='x3')
y3 = FloatSlider(value=end[1], min=origin[1], max=end[1],step=0.01, description='y3')
z3 = FloatSlider(value=end[2], min=origin[2], max=end[2], step=0.01, description='z3')

radio = RadioButtons(options=['above', 'below'], disabled=False)
b = Button(description='Cut')
b.on_click(on_button_clicked)

ui1 = HBox([x1, y1, z1])
ui2 = HBox([x2, y2, z2])
ui3 = HBox([x3, y3, z3])
ui4 = HBox([radio, b])

ui = VBox([ui1, ui2, ui3, ui4])

out = interactive_output(xyz, {'x1':x1,'y1':y1,'z1':z1, 'x2':x2,'y2':y2,'z2':z2, 'x3':x3,'y3':y3,'z3':z3, 'radio':radio})    

display(ui, out)
plot.display()

In [ ]:
fragment = cutted_data[0]
k3d.points(fragment.astype(np.float32), point_size=0.05)

In [ ]:
# POINT HIGHLIGHTER

from ipywidgets import interact, Output
from ipywidgets.widgets import FloatSlider, IntSlider, Button
from ipywidgets.widgets import HBox, interactive_output
from IPython.display import display, clear_output

data = fragment

def xyz(n):
    data_points.colors = np.zeros(data.shape[0], dtype=np.uint32)
    data_points.colors[n] = 0xff0000
    
    data_points.colors = data_points.colors

    
plot = k3d.plot()

data_points = k3d.points(data.astype(np.float32), point_size=0.05, color=0) 
plot += data_points

n = IntSlider(min=0, max=data.shape[0]-1, description='n')

ui = HBox([n])
out = interactive_output(xyz, {'n':n})    

display(ui, out)
plot.display()